# Stock Analysis and Back Testing a Stratergy

**Hypothesis:**

If all 75 ticks in a day is above 50 SMA, Trigger buy @ 3:25 pm. Exit at first instance of close below 50 sma.

**Strategy:**

If the stock price is on rise throughout the day, trigger buy at the end of the day. Hold on to the stock till it is in increasing phase. Exit the stock when it shows the sign of decline.

Not to exit at first decline but instead if it goes lower than the average of last 50 instances.

## Import Liabraries 

In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import yfinance as yf
from pandas_datareader import data as pdr

## Import Stock Data 

- The data is stock price of **Bank Nifty**.

- We have data of **Open, High, Low, Close, SMA20, SMA50, SMA100**.

- The data starts from **July 2011** and goes till **January 2022**.

- The data is present of every **5 minute** interval starting from **9:15 am** and goes upto **3:25 pm**. 

- In total there are **75 intervals** in a day.

In [2]:
nse_data = pd.read_excel("Bank Nifty data 5 min.xlsx", header = 2)

In [3]:
nse_data.head()

,Date,Time,Open,High,Low,Close,SMA20,SMA50,SMA 100
0,2011-07-29,09:15:00,10820.65,10820.65,10758.40,10774.20,NaN,NaN,NaN
1,2011-07-29,09:20:00,10774.20,10812.40,10774.20,10807.95,NaN,NaN,NaN
2,2011-07-29,09:25:00,10807.95,10837.25,10804.35,10837.25,NaN,NaN,NaN
3,2011-07-29,09:30:00,10836.05,10891.45,10835.60,10884.80,NaN,NaN,NaN
4,2011-07-29,09:35:00,10884.80,10892.30,10871.85,10891.65,NaN,NaN,NaN


In [4]:
nse_data = nse_data.fillna(-1)
nse_data.head()

,Date,Time,Open,High,Low,Close,SMA20,SMA50,SMA 100
0,2011-07-29,09:15:00,10820.65,10820.65,10758.40,10774.20,-1.0,-1.0,-1.0
1,2011-07-29,09:20:00,10774.20,10812.40,10774.20,10807.95,-1.0,-1.0,-1.0
2,2011-07-29,09:25:00,10807.95,10837.25,10804.35,10837.25,-1.0,-1.0,-1.0
3,2011-07-29,09:30:00,10836.05,10891.45,10835.60,10884.80,-1.0,-1.0,-1.0
4,2011-07-29,09:35:00,10884.80,10892.30,10871.85,10891.65,-1.0,-1.0,-1.0


In [5]:
nse_data.tail()

,Date,Time,Open,High,Low,Close,SMA20,SMA50,SMA 100
192328,2022-01-24,15:05:00,36874.30,36946.70,36813.45,36946.70,36725.6400,36968.661,37242.978218
192329,2022-01-24,15:10:00,36946.55,37063.65,36929.50,37038.35,36735.3350,36960.372,37237.150000
192330,2022-01-24,15:15:00,37048.60,37073.95,36990.90,37050.30,36747.5000,36953.933,37231.681188
192331,2022-01-24,15:20:00,37047.80,37062.85,36894.25,36905.65,36751.2025,36944.372,37223.801980
192332,2022-01-24,15:25:00,36902.40,36945.05,36808.15,36834.90,36751.3125,36934.724,37215.664356


In [6]:
nse_data.iloc[-1]["Close"]

36834.9

## 1. Find the number of days where all 75 ticks had close value greater than SMA50 

In [7]:
condition_satisfied = condition_not_satisfied = 0

for index in nse_data.index:
    if nse_data["Time"][index] == dt.time(9,15,0):
        counter = 0
    
    if nse_data["Close"][index] >= nse_data["SMA50"][index]:
        counter += 1
    else:
        counter = -1
    
    if nse_data["Time"][index] == dt.time(15,25,0):
        if counter == 75:
            # all 75 ticks satisfied the condition
            condition_satisfied += 1
        else:
            # condition failed
            condition_not_satisfied += 1

In [8]:
"Number of days condition was satisfied: " + str(condition_satisfied)

'Number of days condition was satisfied: 146'

In [9]:
"Number of days condition was satisfied: " + str(condition_not_satisfied)

'Number of days condition was satisfied: 2416'

 ### Inference: This tells us that there were 146 days where the stock price of BANK NIFTY was on rise throughout the day.
 
 i.e. The price of stock at any point of time on that day was greater than the SMA50 (average of price of the last 50 points)

## 2. Find the number of days where all 75 ticks had close value less than SMA50 

In [10]:
condition_satisfied = condition_not_satisfied = 0

for index in nse_data.index:
    if nse_data["Time"][index] == dt.time(9,15,0):
        counter = 0
    
    if nse_data["Close"][index] <= nse_data["SMA50"][index]:
        counter += 1
    else:
        counter = -1
    
    if nse_data["Time"][index] == dt.time(15,25,0):
        if counter == 75:
            # all 75 ticks satisfied the condition
            condition_satisfied += 1
        else:
            # condition failed
            condition_not_satisfied += 1

In [11]:
"Number of days condition was satisfied: " + str(condition_satisfied)

'Number of days condition was satisfied: 105'

In [12]:
"Number of days condition was not satisfied: " + str(condition_not_satisfied)

'Number of days condition was not satisfied: 2457'

 ### Inference: This tells us that there were 105 days where the stock price of BANK NIFTY was on fall throughout the day.
 
 i.e. The price of stock at any point of time on that day was lower than the SMA50 (average of price of the last 50 points)

## 3. Buy the Stock at the end of the day when the Price is at rice and Sell when it starts to decline.


Stratergy:
- Buy at 3:25 pm on the day when all 75 ticks had close value greater than the SMA50.
- Sell at the first instant when the close value is less than the SMA50.
- Find out the money made or lost with this stratergy over last 10 year.


Given 10 lakh rupees at the start of the data  find out how much money was made or lost in this stratergy.

In [13]:
current_money_sma50 = 1000000
buying_mode = True
profit_made_sma50 = []

for index in nse_data.index:
    if buying_mode == True:    
        if nse_data["Time"][index] == dt.time(9,15,0):
            counter = 0

        if nse_data["Close"][index] >= nse_data["SMA50"][index]:
            counter += 1
        else:
            counter = -1

        if nse_data["Time"][index] == dt.time(15,25,0):
            if counter == 75:
                # all 75 ticks satisfied the condition
                # trigger buy
                buy_price = nse_data["Close"][index]
                num_stocks_bought = current_money_sma50//buy_price
                current_money_sma50 = current_money_sma50%buy_price
                buying_mode = False
                continue
    
    if buying_mode == False:
        if nse_data["Close"][index] < nse_data["SMA50"][index]:
            selling_price = nse_data["Close"][index]
            profit_made = (selling_price - buy_price)*num_stocks_bought
            current_money_sma50 += selling_price*num_stocks_bought 
            profit_made_sma50.append(profit_made)
            buying_mode = True

In [14]:
len(profit_made_sma50)

130

In [15]:
"The profit made using this stratergy: " + str(sum(profit_made_sma50))

'The profit made using this stratergy: 809241.2500000012'

In [16]:
if buying_mode == False:
    selling_price = nse_data.iloc[-1]["Close"]
    current_money += selling_price*num_stocks_bought


"The total amount of money at the end of the data: " + str(current_money_sma50)

'The total amount of money at the end of the data: 1809241.25'

# Conclusion

#### Money made with this stratergy would be 8 Lakhs 9 Thousand at an investment of 10 lakh rupees.